# 一、模块

Python所有内置函数：https://docs.python.org/3/library/functions.html

！自己创建模块时要注意命名，不能和Python自带的模块名称冲突。例如，系统自带了sys模块，自己的模块就不可命名为sys.py，否则将无法导入系统自带的sys模块

模块名不要和系统模块名冲突，最好先查看系统是否已存在该模块，检查方法是在Python交互环境执行import abc，若成功则说明系统存在此模块。

# 二、面向对象编程

**1.获取对象信息**

判断对象类型，使用isinstance()函数

In [ ]:
isinstance('a', str)

如果要获得一个对象的所有属性和方法，可以使用dir()函数，它返回一个包含字符串的list，比如，获得一个str对象的所有属性和方法：

In [ ]:
dir('ABC')

如果也想用len(myObj)的话，就自己写一个\_\_len\_\_()方法：

In [ ]:
class MyObject(object):
    def __len__(self):
        return 100

obj = MyObject()
len(obj)

配合getattr()、setattr()以及hasattr()，我们可以直接操作一个对象的状态：

In [ ]:
class MyObject(object):
    def __init__(self):
        self.x = 9
    def power(self):
        return self.x*self.x
obj=MyObject()

In [ ]:
hasattr(obj, 'x') # 有属性'x'吗？

In [ ]:
hasattr(obj, 'y')# 有属性'y'吗？

In [ ]:
setattr(obj, 'y', 19) # 设置一个属性'y'

In [ ]:
hasattr(obj, 'y') # 有属性'y'吗？

In [ ]:
getattr(obj, 'z', 404) # 获取属性'z'，如果不存在，返回默认值404

In [ ]:
hasattr(obj, 'power') # 有属性'power'吗？

**2.class实例动态绑定属性和方法**

In [ ]:
class Student(object):
    pass
s = Student()

In [ ]:
s.name = "Lianyun"#实例绑定一个属性：

In [ ]:
def set_age(self, age):
    self.age = age
from types import MethodType

In [ ]:
s.set_age = MethodType(set_age, s)# 给实例绑定一个方法
s.set_age(25) # 调用实例方法
s.age# 测试结果

但是，给一个实例绑定的方法，对另一个实例是不起作用的：<br/>
为了给所有实例都绑定方法，可以给class绑定方法：

In [ ]:
def set_score(self, score):
    self.score = score
Student.set_score = MethodType(set_score, Student)

In [ ]:
s1 = Student()
s1.set_score(100)
s1.score

In [ ]:
s2 = Student()
s2.set_score(99)
s2.score

定义一个特殊的__slots__变量，来限制该class能添加的属性：

In [ ]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称

In [ ]:
s = Student() # 创建新的实例
s.name = 'Michael' # 绑定属性'name'
s.age = 25 # 绑定属性'age'
s.score = 99 # 绑定属性'score'

使用__slots__要注意，__slots__定义的属性仅对当前类起作用，对继承的子类是不起作用的

**3.@property的使用**

Python内置的@property装饰器就是负责把一个方法变成属性调用的：（把成员函数变为成员变量使用）

In [ ]:
class Person(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2014 - self._birth

birth是可读写属性，而age就是一个只读属性

In [ ]:
p = Person()
p.birth = 1990#实际转化为s.set_birth(1990)
p.birth#实际转化为s.get_birth()

In [ ]:
p.age

In [ ]:
p.age = 100

**4.多重继承**

在设计类的继承关系时，通常，主线都是单一继承下来的，例如，Ostrich继承自Bird。但是，如果需要“混入”额外的功能，通过多重继承就可以实现，比如，让Ostrich除了继承自Bird外，再同时继承Runnable。这种设计通常称之为Mixin。

Python自带的很多库也使用了Mixin。举个例子，Python自带了TCPServer和UDPServer这两类网络服务，而要同时服务多个用户就必须使用多进程或多线程模型，这两种模型由ForkingMixin和ThreadingMixin提供。通过组合，我们就可以创造出合适的服务来。

编写一个多进程模式的TCP服务，定义如下：
~~~
class MyTCPServer(TCPServer, ForkingMixin):
    pass
~~~

编写一个多线程模式的UDP服务，定义如下：
~~~
class MyUDPServer(UDPServer, ThreadingMixin):
    pass
~~~ 

**5.定制类**

~~~
__slots__
__len__()
__str__()
__repr__() 
~~~

In [ ]:
class Someone(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    __repr__ = __str__
    #两者的区别是__str__()返回用户看到的字符串，__repr__()返回开发者看到的字符串，
    #也就是说，__repr__()是为调试服务的
    
print(Someone('Michael'))#调用__str__

one = Someone('Jordan')
print(one)#调用__repr__

~~~
__iter__() 
__getitem__()
__setitem__()
__delitem__()
~~~

如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个__iter__()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的next()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

In [1]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1 # 初始化两个计数器a，b

    def __iter__(self):
        return self # 实例本身就是迭代对象，故返回自己

    def next(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.a > 10: # 退出循环的条件
            raise StopIteration();
        return self.a # 返回下一个值

In [5]:
for n in Fib():
    print(n)

TypeError: iter() returned non-iterator of type 'Fib'

要表现得像list那样按照下标取出元素，需要实现\_\_getitem\_\_()方法：

In [6]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

In [9]:
f = Fib()
print (f[0])
print (f[10])

1
89


如果要支持切片操作，getitem传入的参数要支持切片对象slice

In [10]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int):
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice):
            start = n.start
            stop = n.stop
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

In [11]:
f = Fib()
f[0:5]

[1, 1, 2, 3, 5]

~~~
__getattr__()
~~~

In [12]:
class Student(object):

    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, attr):
        if attr=='score':
            return 99
        if attr=='age':
            return lambda: 25 
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)

In [13]:
s = Student()
s.name

'Michael'

In [14]:
s.score

99

In [15]:
s.age()

25

In [16]:
s.city

AttributeError: 'Student' object has no attribute 'city'

注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，不会在__getattr__中查找。

~~~
__call__()
~~~

In [17]:
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)

In [18]:
s = Student('Michael')
s()#直接对实例进行调用

My name is Michael.


怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有\_\_call()\_\_的类实例：

通过callable()函数，我们就可以判断一个对象是否是“可调用”对象。

In [20]:
callable(s)

True

In [21]:
callable(max)

True

In [22]:
callable([1, 2, 3])

False

**5.元类**

todo